# Tensorflow_ResNet50v1_preprocess
在本脚本中，我们测试了两种预处理方法对ResNet50_v1的影响
1. Slim函数自带的ResNet50预处理方法
2. 论文方法，将输入图像减去ImageNet中的均值进行0归一化

## 结论：
在本测试图片中，我发现第二种方法的预处理要优于Slim自带的预处理方法

================================================================================================ 

### 方法一：Slim函数自带的ResNet50预处理算法：
- 给定短边长度，等比例resize图片。
- 中心切片。
- 减去ImageNet训练集的RGB均值。

In [4]:
import tensorflow as tf
from tensorflow.contrib.slim import nets
slim = tf.contrib.slim
from PIL import Image 
import numpy as np
from preprocessing import vgg_preprocessing

In [5]:
inputs = tf.placeholder("float32", shape=[None, None, 3], name="inputs")
preprocess_inputs = vgg_preprocessing.preprocess_image(inputs, 224, 224, is_training=False)
preprocess_inputs = tf.expand_dims(preprocess_inputs, 0)
with slim.arg_scope(nets.resnet_v1.resnet_arg_scope()):
    net, endpoints = nets.resnet_v1.resnet_v1_50(preprocess_inputs, num_classes=1000, is_training=False)
prob = tf.nn.softmax(net)

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [6]:
sess = tf.InteractiveSession()
variables_to_restore = slim.get_variables_to_restore()
init_fn = slim.assign_from_checkpoint_fn("resnet_v1_50.ckpt", variables_to_restore, ignore_missing_vars=True)
init_fn(sess)

INFO:tensorflow:Restoring parameters from resnet_v1_50.ckpt


In [18]:
im = np.array(Image.open("1.jpg"),dtype=np.float32)

In [19]:
t = sess.run(endpoints["predictions"], feed_dict={inputs: im})
print("find the top1 prediction",np.where(t==np.max(t)))
print("print prob: ",t[0][0][0][386])

find the top1 prediction (array([0], dtype=int64), array([0], dtype=int64), array([0], dtype=int64), array([386], dtype=int64))
print prob:  0.962141


### 方法二：将图像减去ImageNet均值归一化
- 输入图像（0-255） substract ImageNet的均值[123.68, 116.78, 103.94]

In [1]:
import tensorflow as tf
from tensorflow.contrib.slim import nets
slim = tf.contrib.slim
from PIL import Image 
import numpy as np

In [2]:
inputs = tf.placeholder("float32", shape=[None, None, None, 3], name="inputs")
with slim.arg_scope(nets.resnet_v1.resnet_arg_scope()):
    net, endpoints = nets.resnet_v1.resnet_v1_50(inputs, num_classes=1000, is_training=False)
prob = tf.nn.softmax(net)

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [3]:
sess = tf.InteractiveSession()
variables_to_restore = slim.get_variables_to_restore()
init_fn = slim.assign_from_checkpoint_fn("resnet_v1_50.ckpt", variables_to_restore, ignore_missing_vars=True)
init_fn(sess)

INFO:tensorflow:Restoring parameters from resnet_v1_50.ckpt


In [7]:
im = np.array(Image.open("1.jpg").resize((224,224)),dtype=np.float32)
mean = [123.68, 116.78, 103.94] # RGB
im = np.array(im) - mean
im = np.expand_dims(im,0)

In [8]:
t = sess.run(endpoints["predictions"], feed_dict={inputs: im})
print("find the top1 prediction",np.where(t==np.max(t)))
print("print prob: ",t[0][0][0][386])

find the top1 prediction (array([0], dtype=int64), array([0], dtype=int64), array([0], dtype=int64), array([386], dtype=int64))
print prob:  0.971966
